<a href="https://colab.research.google.com/github/IMI1998/zenml/blob/main/zenml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install "zenml[server]"
!zenml integration install sklearn -y
%pip install pyparsing==2.4.2
%pip install zenml



NumExpr defaulting to 2 threads.
⠙ Installing integrations...


In [ ]:
NGROK_TOKEN = "2bDnUfDo5TLvPNJD6s0rgZ66V6m_4Sui54Juz6qyGbsfZJuc6"

In [ ]:
from zenml.environment import Environment

if Environment.in_google_colab():

    !pip install pyngrok
    !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
Setting the repo active workspace to 'default'.
Setting the repo active stack to default.
ZenML repository initialized at /content.
⠦ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.

The local active stack was initialized to 'default'. This local configuration will only take effect 
when you're running ZenML from the initialized repository root, or from a subdirectory. For more 
information on repositories and configurations, please visit 
https://docs.zenml.io/user-guide/starter-guide/understand-stacks.


In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test() -> None:
    "Train and test a sklearn svc classifier on digits"
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train , X_test , y_train , y_test = train_test_split(
        data , digits.target , test_size=0.2 , shuffle=False
    )
    model = SVC(gamma=0.001)
    model.fit(X_train , y_train)
    test_acc = model.score(X_test , y_test)
    print(f"test accuracy: {test_acc}")

train_test()



test accuracy: 0.9583333333333334


In [ ]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

@step
def importer() -> Tuple[
    Annotated[np.ndarray , "X_train"],
    Annotated[np.ndarray , "X_test"],
    Annotated[np.ndarray , "y_train"],
    Annotated[np.ndarray , "y_test"],
]:
    """ Load the digits dataset as nampy arrays."""
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    X_train , X_test , y_train , y_test = train_test_split(
        data , digits.target , test_size=0.2 , shuffle=False
    )
    return X_train , X_test , y_train , y_test


@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
     """Train an sklearn SVC classifier"""
     model = SVC(gamma=0.001)
     model.fit(X_train , y_train)
     return model


@step

def evaluation (
    X_test : np.ndarray,
    y_test: np.ndarray,
    model: ClassifierMixin,
) -> float:
   """ Calculate the test set accuracy of an sklearn model."""
   test_acc = model.score(X_test , y_test)
   print(f"Test accuracy: {test_acc}")
   return test_acc


In [ ]:
from zenml import pipeline

@pipeline
def digits_pipline():
    """Links all the steps together in a pipline"""
    X_train , X_test , y_train , y_test=importer()
    model = svc_trainer(X_train=X_train , y_train=y_train)
    evaluation(X_test=X_test ,y_test=y_test , model=model)

In [ ]:
gigits_svc_pipeline=digits_pipline()

Initiating a new run for the pipeline: digits_pipline.
Reusing registered pipeline version: (version: 2).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.
Using cached version of importer.
Step importer has started.
Using cached version of svc_trainer.
Step svc_trainer has started.
Using cached version of evaluation.
Step evaluation has started.
Pipeline run has finished in 0.935s.


In [ ]:
from zenml.environment import Environment

def start_zenml_dashboard(port=8888):
    if Environment.in_google_colab():
        from pyngrok import ngrok

        public_url = ngrok.connect(port)
        print(f"\x1b[31mIn Colab , use this: {public_url}!\x1b[0m")
        !zenml up --blocking --port {port}

    else:
        !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8888-ef9eff51-0b90-4035-a11d-52ef024a9dd3


Opening tunnel named: http-8888-ef9eff51-0b90-4035-a11d-52ef024a9dd3


INFO:pyngrok.process.ngrok:t=2024-06-22T13:36:02+0000 lvl=info msg=start pg=/api/tunnels id=188b15c4a9371664


t=2024-06-22T13:36:02+0000 lvl=info msg=start pg=/api/tunnels id=188b15c4a9371664


INFO:pyngrok.process.ngrok:t=2024-06-22T13:36:02+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8888-ef9eff51-0b90-4035-a11d-52ef024a9dd3 addr=http://localhost:8888 url=https://0c49-34-48-90-138.ngrok-free.app


...

t=2024-06-22T14:06:18+0000 lvl=info msg="join connections" obj=join id=3d4ff4e3d0c1 l=127.0.0.1:8888 r=[2a01:5ec0:5812:1a43:49c1:297d:4bd:7419]:53973


INFO:pyngrok.process.ngrok:t=2024-06-22T14:06:18+0000 lvl=info msg="join connections" obj=join id=016395f791ef l=127.0.0.1:8888 r=[2a01:5ec0:5812:1a43:49c1:297d:4bd:7419]:53973


t=2024-06-22T14:06:18+0000 lvl=info msg="join connections" obj=join id=016395f791ef l=127.0.0.1:8888 r=[2a01:5ec0:5812:1a43:49c1:297d:4bd:7419]:53973
INFO:     2a01:5ec0:5812:1a43:49c1:297d:4bd:7419:0 - "GET /api/v1/info HTTP/1.1" 200 OK
Got token from cookie
INFO:     2a01:5ec0:5812:1a43:49c1:297d:4bd:7419:0 - "GET /api/v1/settings HTTP/1.1" 200 OK


INFO:pyngrok.process.ngrok:t=2024-06-22T14:06:24+0000 lvl=info msg="join connections" obj=join id=ccea2372a6b4 l=127.0.0.1:8888 r=[2a01:5ec0:5812:1a43:49c1:297d:4bd:7419]:53973


t=2024-06-22T14:06:24+0000 lvl=info msg="join connections" obj=join id=ccea2372a6b4 l=127.0.0.1:8888 r=[2a01:5ec0:5812:1a43:49c1:297d:4bd:7419]:53973
Got token from cookie
INFO:     2a01:5ec0:5812:1a43:49c1:297d:4bd:7419:0 - "PUT /api/v1/settings HTTP/1.1" 200 OK
Got token from cookie
INFO:     2a01:5ec0:5812:1a43:49c1:297d:4bd:7419:0 - "GET /api/v1/settings HTTP/1.1" 200 OK
Got token from cookie
INFO:     2a01:5ec0:5812:1a43:49c1:297d:4bd:7419:0 - "PUT /api/v1/settings HTTP/1.1" 200 OK
Got token from cookie
INFO:     2a01:5ec0:5812:1a43:49c1:297d:4bd:7419:0 - "GET /api/v1/settings HTTP/1.1" 200 OK
